In [ ]:
import pandas as pd
ml=pd.read_csv('../input/phishing-dataset-for-machine-learning/Phishing_Legitimate_full.csv')
ml.head()

In [ ]:
ml.info()

In [ ]:
unique=ml.nunique().sort_values(ascending=True).reset_index()
unique

In [ ]:
categorical_features=unique[(unique[0] <10)]['index'].tolist()
categorical_features.remove('CLASS_LABEL')
high_cardinality_features=unique[(unique[0] >= 10) & (unique[0] <=20)]['index'].tolist()
numeric_features=unique[(unique[0] > 20)]['index'].tolist()
numeric_features.remove('id')

In [ ]:
len(categorical_features),len(high_cardinality_features),len(numeric_features),(len(categorical_features)+len(high_cardinality_features)+len(numeric_features))

In [ ]:
data = ml.sample(frac=0.8, random_state=42)
data_unseen = ml.drop(data.index)
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)
print('Data for Modeling: ' + str(data.shape))

In [ ]:
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

In [ ]:
!pip install pycaret

In [ ]:
from pycaret.classification import *

In [ ]:
ml.CLASS_LABEL.value_counts()

In [ ]:
phishing = setup(data = data, target = 'CLASS_LABEL', session_id=42,
                  normalize = True, 
                  transformation = True, 
                  log_experiment = True,
                  handle_unknown_categorical = True, 
                  unknown_categorical_method = 'most_frequent',
                  remove_multicollinearity = True, #rop one of the two features that are highly correlated with each other
                  ignore_low_variance = True,#all categorical features with statistically insignificant variances are removed from the dataset.
                  combine_rare_levels = True,# all levels in categorical features below the threshold defined in rare_level_threshold param are combined together as a single level
                  numeric_imputation='median',
                  categorical_imputation='mode',

                ignore_features=['id'],
                 high_cardinality_features=high_cardinality_features,
                 numeric_features=numeric_features
                )

In [ ]:
lightgbm = create_model('lightgbm')

In [ ]:
print(lightgbm)

In [ ]:
tuned_lightgbm = tune_model(lightgbm,optimize = 'AUC')

In [ ]:
plot_model(tuned_lightgbm, plot = 'auc')

In [ ]:
plot_model(tuned_lightgbm, plot = 'pr')

In [ ]:
plot_model(tuned_lightgbm, plot='feature')

In [ ]:
plot_model(tuned_lightgbm, plot = 'confusion_matrix')

In [ ]:
evaluate_model(tuned_lightgbm)

In [ ]:
predict_model(tuned_lightgbm)

In [ ]:
unseen_predictions = predict_model(tuned_lightgbm, data=data_unseen)
unseen_predictions.head()

In [ ]:
print("Confidence Score :   {}".format(round(unseen_predictions.Score.mean(),2)))#Confidence Score